In [1]:
import pandas as pd
import numpy as np


In [2]:
headers = pd.read_csv('./data/car.meta.txt',sep = ':', header = None)[0]
headers = list(headers)
headers

['buying', 'maint', 'doors', 'persons', 'lug_boot', 'safety', 'class']

In [3]:
train_df = pd.read_csv('./data/car.train.txt', header = None)
train_df

,0,1,2,3,4,5,6
0,med,med,3,more,small,low,unacc
1,high,low,4,4,med,high,acc
2,vhigh,vhigh,3,more,big,med,unacc
3,high,vhigh,3,4,big,med,unacc
4,med,low,3,more,big,high,vgood
...,...,...,...,...,...,...,...
1395,high,vhigh,4,4,big,med,unacc
1396,vhigh,med,4,2,small,med,unacc
1397,high,vhigh,3,more,small,low,unacc
1398,med,low,4,more,small,med,acc


In [4]:
train_df.columns = headers

train_df

,buying,maint,doors,persons,lug_boot,safety,class
0,med,med,3,more,small,low,unacc
1,high,low,4,4,med,high,acc
2,vhigh,vhigh,3,more,big,med,unacc
3,high,vhigh,3,4,big,med,unacc
4,med,low,3,more,big,high,vgood
...,...,...,...,...,...,...,...
1395,high,vhigh,4,4,big,med,unacc
1396,vhigh,med,4,2,small,med,unacc
1397,high,vhigh,3,more,small,low,unacc
1398,med,low,4,more,small,med,acc


In [5]:
test_df = pd.read_csv('./data/car.test.txt', header = None)
# test_df.columns = headers

In [6]:
print(test_df.head(30))

        0      1      2     3      4     5      6
0   vhigh    med  5more     4    med   low  unacc
1    high    med      3     2  small  high  unacc
2   vhigh    med      2     4    big  high    acc
3   vhigh   high      4     2    big   low  unacc
4   vhigh  vhigh      3     4    big   med  unacc
5     med   high      3  more  small   low  unacc
6   vhigh  vhigh      2     4    med  high  unacc
7     med    med  5more     4    big   med    acc
8     low  vhigh      4  more  small  high    acc
9     low    med      3     2    big   low  unacc
10    med  vhigh      2     4  small   med  unacc
11   high   high      3     2    med   low  unacc
12   high   high      3  more  small   med  unacc
13  vhigh   high      2     2  small   low  unacc
14   high  vhigh      4     4  small   med  unacc
15    med  vhigh      3     2    med   med  unacc
16  vhigh    low      3  more    med   med    acc
17    med  vhigh      2  more    big   med    acc
18    med   high      4  more    med   low  unacc


## Handcoding algorithm

In [7]:
ind_col_counts = {}
col_dict = {}

for ele in train_df['class'].unique():
    ind_count = np.asarray(train_df[train_df['class']==ele]['class']).size
    ind_col_counts[ele] = ind_count
col_dict = ind_col_counts

In [8]:
# class
# class_probabilities = pd.DataFrame(col_dict)
prob_col_dict = {}
for key in col_dict.keys():
    prob_col_dict[key] = (col_dict[key]/train_df['class'].shape[0])


In [9]:
prob_col_dict

{'unacc': 0.7085714285714285,
 'acc': 0.21785714285714286,
 'vgood': 0.035,
 'good': 0.03857142857142857}

In [10]:
#buying
probs_list = {}
for header in headers:
    ind_df = pd.DataFrame(index = train_df['class'].unique(), columns = train_df[header].unique())
    for row in train_df['class'].unique():
        total_vals = train_df[train_df['class'] == row].shape[0]
        for ele in train_df[header].unique():
            unacc_med = train_df[(train_df['class'] == row) & (train_df[header] == ele)].shape[0]
            ind_df[ele][row] = unacc_med / total_vals
    probs_list[header] = ind_df
for key in probs_list.keys():
    print('\n', key, ' probabilities----\n', probs_list[key])


 buying  probabilities----
             med      high     vhigh       low
unacc  0.221774  0.268145  0.300403  0.209677
acc    0.304918  0.281967  0.183607  0.229508
vgood  0.408163       0.0       0.0  0.591837
good    0.37037       0.0       0.0   0.62963

 maint  probabilities----
             med       low     vhigh      high
unacc  0.220766  0.222782  0.300403  0.256048
acc    0.298361   0.24918  0.180328  0.272131
vgood  0.346939  0.469388       0.0  0.183673
good   0.314815  0.685185       0.0       0.0

 doors  probabilities----
               3         4         2     5more
unacc  0.243952  0.238911  0.268145  0.248992
acc    0.265574   0.27541  0.196721  0.262295
vgood   0.22449  0.326531  0.163265  0.285714
good   0.240741  0.277778  0.203704  0.277778

 persons  probabilities----
            more         4        2
unacc  0.265121  0.261089  0.47379
acc    0.472131  0.527869      0.0
vgood  0.591837  0.408163      0.0
good   0.537037  0.462963      0.0

 lug_boot  probabil

In [11]:
def getProbabilityItems(header_list = headers, feature_list = ['high', 'vhigh', '5more', '4', 'big', 'med'], pred_var='acc'):
    selection_lst = []
    for item_no in range(6):
        selection_lst.append(probs_list[header_list[item_no]][feature_list[item_no]][pred_var])
    selection_lst.append(prob_col_dict[pred_var])
    return selection_lst

In [12]:
def productOfItems(item_list):
    product = 1
    for item in item_list:
        product = product * item
    return product


In [13]:
def getMaxIndex(productsOfClass):
    return productsOfClass.index(max(productsOfClass))

In [14]:
def predict(header_list = headers[:6], feature_list = ['vhigh', 'med', '2', '4', 'big', 'high']):
    productsOfClass = []
    for pred_item in prob_col_dict.keys():
        probabs_lst = getProbabilityItems(feature_list = feature_list, pred_var = pred_item)
        productsOfClass.append(productOfItems(probabs_lst))
    prediction = list(prob_col_dict.keys())[getMaxIndex(productsOfClass)]
    return prediction

In [15]:
predictions = []
for row in range(test_df.shape[0]):
    feature_list = test_df.iloc[row]
    feature_list = list(feature_list[:6])
#     print(feature_list)
    predictions.append(predict(feature_list = feature_list))

In [16]:
print(predictions)

['unacc', 'unacc', 'acc', 'unacc', 'unacc', 'unacc', 'unacc', 'acc', 'unacc', 'unacc', 'unacc', 'unacc', 'unacc', 'unacc', 'unacc', 'unacc', 'unacc', 'unacc', 'unacc', 'unacc', 'acc', 'unacc', 'unacc', 'unacc', 'acc', 'unacc', 'unacc', 'unacc', 'unacc', 'unacc', 'acc', 'unacc', 'unacc', 'unacc', 'unacc', 'acc', 'unacc', 'acc', 'unacc', 'unacc', 'unacc', 'unacc', 'unacc', 'acc', 'unacc', 'unacc', 'unacc', 'unacc', 'good', 'unacc', 'unacc', 'unacc', 'unacc', 'unacc', 'unacc', 'unacc', 'unacc', 'unacc', 'unacc', 'acc', 'unacc', 'unacc', 'unacc', 'good', 'unacc', 'acc', 'unacc', 'unacc', 'unacc', 'unacc', 'unacc', 'acc', 'acc', 'acc', 'unacc', 'unacc', 'unacc', 'unacc', 'unacc', 'acc', 'acc', 'unacc', 'acc', 'unacc', 'vgood', 'unacc', 'acc', 'unacc', 'unacc', 'unacc', 'unacc', 'acc', 'vgood', 'unacc', 'acc', 'acc', 'unacc', 'unacc', 'unacc', 'unacc', 'unacc', 'unacc', 'unacc', 'acc', 'unacc', 'unacc', 'unacc', 'acc', 'good', 'unacc', 'unacc', 'acc', 'unacc', 'unacc', 'unacc', 'unacc', 'una

In [34]:
no_of_right_predictions = test_df[test_df[test_df.iloc[0].shape[0] - 1] == predictions].shape[0]
total_class = test_df.shape[0]
print(total_class)

328


In [ ]:
count = 0
for i in range(len(predictions)):
#     predictions[i] == 'unacc' 
    if(predictions[i] == 'unacc'):
        count = count + 1

In [ ]:
print(count)